<a href="https://colab.research.google.com/github/FozilovDiyorbek/Qilingan-loyihalar/blob/main/Airvays_Ticket_Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Necessary Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.metrics import  mean_absolute_error, r2_score, mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor

In [3]:
df = pd.read_csv('/content/train_data.csv')
df

,id,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,1,Vistara,UK-810,Bangalore,Early_Morning,one,Night,Mumbai,Economy,14.25,21,7212
1,2,SpiceJet,SG-5094,Hyderabad,Evening,zero,Night,Kolkata,Economy,1.75,7,5292
2,3,Vistara,UK-846,Bangalore,Morning,one,Evening,Delhi,Business,9.58,5,60553
3,4,Vistara,UK-706,Kolkata,Morning,one,Evening,Hyderabad,Economy,6.75,28,5760
4,5,Indigo,6E-5394,Chennai,Early_Morning,zero,Morning,Mumbai,Economy,2.00,4,10712
...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19996,Indigo,6E-6178,Bangalore,Night,one,Early_Morning,Mumbai,Economy,7.92,45,3153
19996,19997,AirAsia,I5-582,Kolkata,Morning,one,Afternoon,Delhi,Economy,5.83,24,3911
19997,19998,Vistara,UK-832,Chennai,Early_Morning,two_or_more,Evening,Bangalore,Economy,35.33,17,14822
19998,19999,Vistara,UK-996,Mumbai,Evening,one,Morning,Bangalore,Economy,16.33,21,6450


In [4]:
test_data = pd.read_csv('/content/test_data (1).csv')
test_data

,id,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left
0,1,Air_India,AI-765,Kolkata,Evening,one,Night,Delhi,Business,28.25,2
1,2,Vistara,UK-747,Delhi,Early_Morning,one,Night,Mumbai,Business,13.83,34
2,3,Air_India,AI-570,Mumbai,Early_Morning,zero,Early_Morning,Chennai,Business,2.00,30
3,4,AirAsia,I5-974,Hyderabad,Night,one,Late_Night,Delhi,Economy,5.17,26
4,5,Air_India,AI-770,Kolkata,Night,one,Afternoon,Mumbai,Economy,16.33,35
...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,Air_India,AI-768,Kolkata,Afternoon,one,Morning,Bangalore,Business,17.42,15
4996,4997,Indigo,6E-6214,Kolkata,Morning,zero,Afternoon,Mumbai,Economy,3.00,40
4997,4998,Air_India,AI-402,Kolkata,Morning,one,Night,Mumbai,Business,11.17,37
4998,4999,Air_India,AI-673,Mumbai,Early_Morning,one,Night,Hyderabad,Business,13.33,38


In [5]:
solving_data = pd.read_csv('/content/sample_solution (1).csv')
solving_data

,id,price
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0
...,...,...
4995,4996,0
4996,4997,0
4997,4998,0
4998,4999,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                20000 non-null  int64  
 1   airline           20000 non-null  object 
 2   flight            20000 non-null  object 
 3   source_city       20000 non-null  object 
 4   departure_time    20000 non-null  object 
 5   stops             20000 non-null  object 
 6   arrival_time      20000 non-null  object 
 7   destination_city  20000 non-null  object 
 8   class             20000 non-null  object 
 9   duration          20000 non-null  float64
 10  days_left         20000 non-null  int64  
 11  price             20000 non-null  int64  
dtypes: float64(1), int64(3), object(8)
memory usage: 1.8+ MB


In [7]:
df.describe()

,id,duration,days_left,price
count,20000.000000,20000.000000,20000.000000,20000.000000
mean,10000.500000,12.177627,25.924150,20960.281700
std,5773.647028,7.157944,13.624874,22775.459535
min,1.000000,0.830000,1.000000,1105.000000
25%,5000.750000,6.830000,14.000000,4783.000000
50%,10000.500000,11.250000,26.000000,7425.000000
75%,15000.250000,16.080000,38.000000,42521.000000
max,20000.000000,38.580000,49.000000,114523.000000


In [8]:
num_cols = ['duration', 'days_left']
transformer = ColumnTransformer([
    ('scaler', StandardScaler(), num_cols)
])

def data_converter(data, cat_cols, num_part=None):

    prepared = pd.DataFrame({}, index = data.index)
    flight = LabelEncoder().fit_transform(data['flight'])
    prepared["fligt"] = StandardScaler().fit_transform(flight.reshape(-1,1)) # so'ng standardlashtiramiz
    for i in range(len(cat_cols)):
        col = pd.get_dummies(data[cat_cols[i]])
        col = col.astype(np.int8)
        prepared = pd.merge(prepared, col, left_index=True, right_index=True)
    if num_part is not None:
        return pd.merge(prepared, num_part, left_index=True, right_index=True)
    else: return prepared

# string columns
cat_cols = ["airline", "source_city", "departure_time", "stops", "arrival_time", "destination_city", "class"]

# number cols
num_part = df[["duration", "days_left", "price","id"]]

# standardization
num_part.loc[:, num_cols] = transformer.fit_transform(num_part)

prepared_df= data_converter(df, cat_cols, num_part)

prepared_df.head()

,fligt,AirAsia,Air_India,GO_FIRST,Indigo,SpiceJet,Vistara,Bangalore_x,Chennai_x,Delhi_x,...,Delhi_y,Hyderabad_y,Kolkata_y,Mumbai_y,Business,Economy,duration,days_left,price,id
0,0.869853,0,0,0,0,0,1,1,0,0,...,0,0,0,1,0,1,0.289528,-0.361418,7212,1
1,0.627668,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,1,-1.456827,-1.388976,5292,2
2,0.955168,0,0,0,0,0,1,1,0,0,...,1,0,0,0,1,0,-0.362910,-1.535770,60553,3
3,0.806555,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,1,-0.758285,0.152361,5760,4
4,-1.714372,0,0,0,1,0,0,0,1,0,...,0,0,0,1,0,1,-1.421900,-1.609167,10712,5


In [21]:
X = prepared_df.drop('price', axis=1)
y = prepared_df['price'].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.04, random_state=42)

In [22]:
tree_model = DecisionTreeRegressor()
tree_model.fit(X_train, y_train)

DecisionTreeRegressor()

In [23]:
 y_predict = tree_model.predict(X_test)

 r2 = r2_score(y_test, y_predict)
 mae = mean_absolute_error(y_test, y_predict)
 mse = mean_squared_error(y_test, y_predict)
 print("Model aniqligi=", r2)
 print("MAE=", mae)
 print("RMSE=", np.sqrt(mse))

Model aniqligi= 0.966846505351852
MAE= 1857.04875
RMSE= 4197.671549948853


In [24]:
RF_model = RandomForestRegressor()
RF_model.fit(X_train, y_train)

RandomForestRegressor()

In [25]:
 y_predict = RF_model.predict(X_test)

 r2 = r2_score(y_test, y_predict)
 mae = mean_absolute_error(y_test, y_predict)
 mse = mean_squared_error(y_test, y_predict)
 print("Model aniqligi=", r2)
 print("MAE=", mae)
 print("RMSE=", np.sqrt(mse))

Model aniqligi= 0.9836529017528516
MAE= 1542.23595
RMSE= 2947.570040128648


In [26]:
xgb_model = XGBRegressor()
xgb_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [27]:
 y_predict = xgb_model.predict(X_test)

 r2 = r2_score(y_test, y_predict)
 mae = mean_absolute_error(y_test, y_predict)
 mse = mean_squared_error(y_test, y_predict)
 print("Model aniqligi=", r2)
 print("MAE=", mae)
 print("RMSE=", np.sqrt(mse))

Model aniqligi= 0.9817099962187301
MAE= 1871.6734283447265
RMSE= 3117.817532059991


The best model RANDOM *FOREST*

In [28]:
num_cols = ['duration', 'days_left']
transformer = ColumnTransformer([
    ('scaler', StandardScaler(), num_cols)
])

def data_converter(data, cat_cols, num_part=None):

    prepared = pd.DataFrame({}, index = data.index)
    flight = LabelEncoder().fit_transform(data['flight'])
    prepared["fligt"] = StandardScaler().fit_transform(flight.reshape(-1,1)) # so'ng standardlashtiramiz
    for i in range(len(cat_cols)):
        col = pd.get_dummies(data[cat_cols[i]])
        col = col.astype(np.int8)
        prepared = pd.merge(prepared, col, left_index=True, right_index=True)
    if num_part is not None:
        return pd.merge(prepared, num_part, left_index=True, right_index=True)
    else: return prepared

# string columns
cat_cols = ["airline", "source_city", "departure_time", "stops", "arrival_time", "destination_city", "class"]

# number cols
num_part = test_data[["duration", "days_left", "id"]]

# standardization
num_part.loc[:, num_cols] = transformer.fit_transform(num_part)

prepared_test_data= data_converter(test_data, cat_cols, num_part)

prepared_test_data.head()

,fligt,AirAsia,Air_India,GO_FIRST,Indigo,SpiceJet,Vistara,Bangalore_x,Chennai_x,Delhi_x,...,Chennai_y,Delhi_y,Hyderabad_y,Kolkata_y,Mumbai_y,Business,Economy,duration,days_left,id
0,-0.597716,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,2.179304,-1.754111,1
1,0.830082,0,0,0,0,0,1,0,0,1,...,0,0,0,0,1,1,0,0.205481,0.583184,2
2,-0.788338,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,-1.413821,0.291022,3
3,0.415198,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,-0.979909,-0.001139,4
4,-0.582765,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0.547683,0.656225,5


In [29]:
 y_predict_solving = RF_model.predict(prepared_test_data)

In [31]:
solving_data['price'] = y_predict_solving

In [32]:
solving_data.to_csv('Aviachipta narhini bashorat qilish.csv', index=False )